In [1]:
import numpy as np
import pandas as pd
import os,sys

In [2]:
def effDictFromFile(fileName,yLabel):
    
    fileDict = {'mass (GeV)' : [], 'lifetime (ns)' : [], 'width (GeV)' : []}
    fileDict[yLabel] = []
    
    # Split file by lifetime:
    with open(fileName,'r') as f:
        blocks = f.read().split('#: Lifetime')[1:]
        for b in blocks:
            b = b.split('\n')
            lifetime = b[0].split(',')[1].replace('ns','')
            if lifetime.strip().lower() == 'stable':
                lifetime = np.inf
                width = 0.0
            else:
                lifetime = eval(lifetime)
                width = 6.582e-25/(lifetime*1e-9)
            for row in b[2:]:
                row = row.strip()
                if not row:
                    continue
                if row[0] == "#":
                    continue
                mass,yval = row.split(',')
                try:
                    yval = eval(yval)
                except:
                    yval = 0.0
                mass = eval(mass)
                fileDict['mass (GeV)'].append(mass)
                fileDict['lifetime (ns)'].append(lifetime)
                fileDict['width (GeV)'].append(width)
                fileDict[yLabel].append(yval)   
    
    return fileDict

def limitDictFromFile(inputFile):
    
    lifetime = inputFile.split('Chargino')[1].split('.csv')[0]
    lifetime = lifetime.replace('ns','')
    if lifetime.strip().lower() == 'stable':
        lifetime = np.inf
        width = 0.0
    else:
        lifetime = eval(lifetime)
        width = 6.582e-25/(lifetime*1e-9)

    fileDict = {'mass (GeV)' : [], 'lifetime (ns)' : [], 'width (GeV)' : [], 
                'Expected Limit (pb)' : [], 'Observed Limit (pb)' : [], 'Signal Cross-Section (pb)' : []}

    # Split file by lifetime:
    with open(inputFile,'r') as f:
        blocks = f.read().split('Chargino Mass [TeV]')[1:]
        for b in blocks:
            b = b.split('\n')
            # Select correct label corresponding to block
            for yLabel in fileDict:
                if yLabel[:10].lower() in b[0].lower():
                    break
            for row in b[1:]:
                row = row.strip()
                if not row:
                    continue
                if row[0] == "#":
                    continue
                mass,yval = row.split(',')[:2]
                try:
                    yval = eval(yval)
                except:
                    yval = 0.0
                mass = eval(mass)*1000.0
                if mass not in fileDict['mass (GeV)']:
                    fileDict['mass (GeV)'].append(mass)
                    fileDict['lifetime (ns)'].append(lifetime)
                    fileDict['width (GeV)'].append(width)
                fileDict[yLabel].append(yval)   
                
    return fileDict

### Acceptance info

In [3]:
inputFile = './HEPData-ins2080541-v1-csv/Acceptance,Chargino.csv'
# Create dict to store information:
accDict = effDictFromFile(inputFile,yLabel = 'acceptance')
accDF = pd.DataFrame(accDict)            
accDF

,mass (GeV),lifetime (ns),width (GeV),acceptance
0,90.0,0.2,3.291000e-15,0.009
1,200.0,0.2,3.291000e-15,0.004
2,250.0,0.2,3.291000e-15,0.000
3,300.0,0.2,3.291000e-15,0.002
4,350.0,0.2,3.291000e-15,0.000
...,...,...,...,...
103,1200.0,inf,0.000000e+00,0.842
104,1300.0,inf,0.000000e+00,0.853
105,1400.0,inf,0.000000e+00,0.829
106,1500.0,inf,0.000000e+00,0.867


### Event-level efficiency

In [4]:
inputFile = './HEPData-ins2080541-v1-csv/Event-levelefficiency,Chargino.csv'
evteffDict = effDictFromFile(inputFile,yLabel = 'event eff')
# Define data frame
evteffDF = pd.DataFrame(evteffDict)            
evteffDF

,mass (GeV),lifetime (ns),width (GeV),event eff
0,90.0,0.2,3.291000e-15,0.423
1,200.0,0.2,3.291000e-15,0.423
2,250.0,0.2,3.291000e-15,0.000
3,300.0,0.2,3.291000e-15,0.402
4,350.0,0.2,3.291000e-15,0.000
...,...,...,...,...
103,1200.0,inf,0.000000e+00,0.202
104,1300.0,inf,0.000000e+00,0.208
105,1400.0,inf,0.000000e+00,0.205
106,1500.0,inf,0.000000e+00,0.212


### SR-Inclusive Low efficiency

In [5]:
inputFile = './HEPData-ins2080541-v1-csv/Efficiency,SR-Inclusive_Low,Chargino.csv'
srlowDict = effDictFromFile(inputFile,yLabel = 'SR-Inclusive_Low eff')
# Define data frame
srlowDF = pd.DataFrame(srlowDict)            
srlowDF

,mass (GeV),lifetime (ns),width (GeV),SR-Inclusive_Low eff
0,90.0,0.2,3.291000e-15,0.000
1,200.0,0.2,3.291000e-15,0.006
2,250.0,0.2,3.291000e-15,0.000
3,300.0,0.2,3.291000e-15,0.006
4,350.0,0.2,3.291000e-15,0.000
...,...,...,...,...
103,1200.0,inf,0.000000e+00,0.030
104,1300.0,inf,0.000000e+00,0.030
105,1400.0,inf,0.000000e+00,0.029
106,1500.0,inf,0.000000e+00,0.030


### SR-Inclusive Low efficiency

In [6]:
inputFile = './HEPData-ins2080541-v1-csv/Efficiency,SR-Inclusve_High,Chargino.csv'
srhighDict = effDictFromFile(inputFile,yLabel = 'SR-Inclusive_High eff')
# Define data frame
srhighDF = pd.DataFrame(srhighDict)            
srhighDF

,mass (GeV),lifetime (ns),width (GeV),SR-Inclusive_High eff
0,90.0,0.2,3.291000e-15,0.000
1,200.0,0.2,3.291000e-15,0.000
2,250.0,0.2,3.291000e-15,0.000
3,300.0,0.2,3.291000e-15,0.000
4,350.0,0.2,3.291000e-15,0.000
...,...,...,...,...
103,1200.0,inf,0.000000e+00,0.069
104,1300.0,inf,0.000000e+00,0.077
105,1400.0,inf,0.000000e+00,0.076
106,1500.0,inf,0.000000e+00,0.087


### Cross-section limits

In [7]:
allDicts = []
inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Chargino10ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Chargino30ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Chargino4ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,CharginoStable.csv'
allDicts.append(limitDictFromFile(inputFile))

xsecAll = {}
for d in allDicts:
    for k,vals in d.items():
        if not k in xsecAll:
            xsecAll[k] = []
        xsecAll[k] += vals

In [8]:
xsecDF = pd.DataFrame(xsecAll)
xsecDF

,mass (GeV),lifetime (ns),width (GeV),Expected Limit (pb),Observed Limit (pb),Signal Cross-Section (pb)
0,400.0,10.0,6.582000e-17,0.003460,0.006470,0.175000
1,500.0,10.0,6.582000e-17,0.002400,0.003180,0.065900
2,600.0,10.0,6.582000e-17,0.001670,0.001490,0.028500
3,700.0,10.0,6.582000e-17,0.001260,0.001000,0.013200
4,800.0,10.0,6.582000e-17,0.001250,0.001160,0.006580
5,900.0,10.0,6.582000e-17,0.001040,0.001340,0.003420
6,1000.0,10.0,6.582000e-17,0.000899,0.001500,0.001820
7,1100.0,10.0,6.582000e-17,0.000867,0.001730,0.001010
8,1200.0,10.0,6.582000e-17,0.000760,0.001720,0.000562
9,1300.0,10.0,6.582000e-17,0.000696,0.001640,0.000322


In [9]:
allDFs = [accDF,evteffDF,srlowDF,srhighDF,xsecDF]

df = accDF
for dd in allDFs[1:]:
    df = df.merge(dd,on=['mass (GeV)','lifetime (ns)','width (GeV)'],how='outer')

In [10]:
df

,mass (GeV),lifetime (ns),width (GeV),acceptance,event eff,SR-Inclusive_Low eff,SR-Inclusive_High eff,Expected Limit (pb),Observed Limit (pb),Signal Cross-Section (pb)
0,90.0,0.2,3.291000e-15,0.009,0.423,0.000,0.000,NaN,NaN,NaN
1,200.0,0.2,3.291000e-15,0.004,0.423,0.006,0.000,NaN,NaN,NaN
2,250.0,0.2,3.291000e-15,0.000,0.000,0.000,0.000,NaN,NaN,NaN
3,300.0,0.2,3.291000e-15,0.002,0.402,0.006,0.000,NaN,NaN,NaN
4,350.0,0.2,3.291000e-15,0.000,0.000,0.000,0.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
103,1200.0,inf,0.000000e+00,0.842,0.202,0.030,0.069,0.000671,0.00172,0.000562
104,1300.0,inf,0.000000e+00,0.853,0.208,0.030,0.077,0.000568,0.00154,0.000322
105,1400.0,inf,0.000000e+00,0.829,0.205,0.029,0.076,0.000546,0.00169,0.000184
106,1500.0,inf,0.000000e+00,0.867,0.212,0.030,0.087,0.000504,0.00137,0.000106


In [11]:
pickleFile = 'charginoData.pcl'
print('Saving to',pickleFile)
df.to_pickle(pickleFile)

Saving to charginoData.pcl


In [14]:
massWidth = set([])
for irow,row in df.iterrows():
    m,w = row['mass (GeV)'],row['lifetime (ns)']
    massWidth.add((m,w))
massWidth = sorted(list(massWidth))
print(len(massWidth))
print(massWidth)

108
[(90.0, 0.2), (90.0, 1.0), (90.0, 4.0), (90.0, 10.0), (90.0, 30.0), (90.0, inf), (200.0, 0.2), (200.0, 1.0), (200.0, 4.0), (200.0, 10.0), (200.0, 30.0), (200.0, inf), (250.0, 0.2), (250.0, 1.0), (250.0, 4.0), (250.0, 10.0), (250.0, 30.0), (250.0, inf), (300.0, 0.2), (300.0, 1.0), (300.0, 4.0), (300.0, 10.0), (300.0, 30.0), (300.0, inf), (350.0, 0.2), (350.0, 1.0), (350.0, 4.0), (350.0, 10.0), (350.0, 30.0), (350.0, inf), (400.0, 0.2), (400.0, 1.0), (400.0, 4.0), (400.0, 10.0), (400.0, 30.0), (400.0, inf), (500.0, 0.2), (500.0, 1.0), (500.0, 4.0), (500.0, 10.0), (500.0, 30.0), (500.0, inf), (600.0, 0.2), (600.0, 1.0), (600.0, 4.0), (600.0, 10.0), (600.0, 30.0), (600.0, inf), (700.0, 0.2), (700.0, 1.0), (700.0, 4.0), (700.0, 10.0), (700.0, 30.0), (700.0, inf), (800.0, 0.2), (800.0, 1.0), (800.0, 4.0), (800.0, 10.0), (800.0, 30.0), (800.0, inf), (900.0, 0.2), (900.0, 1.0), (900.0, 4.0), (900.0, 10.0), (900.0, 30.0), (900.0, inf), (1000.0, 0.2), (1000.0, 1.0), (1000.0, 4.0), (1000.0, 1

In [16]:
1300./13

100.0